In [1]:
import os
import re
import csv 

## GET THE PATHS FOR ALL THE PATIENTS 
def get_files(path_root, patients, types):
    # in types is signal that you want to extract. e.g. intrinsic,underlying 
    all_files = []
    new_path = []
    for i in patients:
        new_path = path_root+i
        new_path = new_path +'/'
        folder_files = os.listdir(new_path)
        for j in range(0,len(folder_files)):
            file_path =  new_path + folder_files[j]
            for file in types:
                a = file_path.find(file)
                if a >0:
                    all_files.append(file_path)
    return all_files
def read_data(all_files,signal_type):
    patients_list = []
    data = []
    data2 = []
    c=1;

    for i in range(0,len(all_files)):
        new_path = all_files[i] + signal_type
        patients_no = int(re.findall(r'\d+', new_path)[1])  # Extract the patient id for each signal
        patients_list.append(patients_no)
        signals = {}
        
        X = []  # The temporary location of the signal
        with open(new_path,'r') as f:
            signal = csv.reader(f, delimiter=',')
            for ROWS in signal:
                X.append(float(ROWS[0]))
        #signals = {"Patient %i" %patients_no : X};
        #df = pd.DataFrame(signals)
        #data.append(signals)  # 
        
       # signals = pd.DataFrame(data)
        #signals = signals.apply(lambda x: pd.Series(x.dropna().values))  # signals organised in a df with the dropped N/A
        
        data2.append(X)  # a list with all the signals
    return data2,patients_list



In [2]:
import numpy as np
# def normalise_func(signal, a, b):
#     # Normalize signals between -1 and 1 
#     # solving system of linear equations one can find the coefficients
#     A = np.min(signal)
#     B = np.max(signal)
#     C = (a-b)/(A-B)
#     k = (C*A - a)/C
#     return (signal-k)*C

#Normalize signals between -1 and 1 
def normalise(filtered_signals,a,b):
    normalized_signals = []
    for i in range (0,len(filtered_signals)):
        signal = filtered_signals[i]
        A = np.min(signal)
        B = np.max(signal)
        C = (a-b)/(A-B)
        k = (C*A - a)/C
        normalized = (signal-k)*C

        normalized_signals.append(normalized)
    return normalized_signals

# def normalise(filtered_signals):
#     normalized_signals = []
#     for i in range (0,len(filtered_signals)):
#         signal = filtered_signals[i]
        
#         signal_mean = np.mean(signal)
#         signal_std = np.std(signal)
#         normalized = (signal - signal_mean) / signal_std

#         normalized_signals.append(normalized)
#     return normalized_signals


In [3]:
## FILTERING THE ECG SIGNALS   
import scipy
from scipy.signal import savgol_filter
def Savitzky_Golay(signals, fs):
    signals_filtered = []
    
    for i in range(0,len(signals)):
        # Savitzky-Golay filter
        y_filtered = savgol_filter(signals[i], 11, 2)
        signals_filtered.append(y_filtered) 
    return  signals_filtered

# Down-sampling

In [4]:
def down_sample(baseline_segments): #paced_segments
    from scipy import signal
    for i in range(0,len(baseline_segments)):
        baseline_segments[i] = signal.decimate(baseline_segments[i], 4)  # down sample signals to 250

    return baseline_segments

In [5]:
# cut the noisy signals in multiple signals 

def cut_noise(signals_filtered,patients_baseline,labels):
    signals_new = []
    patients_list = []
    labels_list = []
    for i in range(0,len(signals_filtered)):
        signals_new.append(signals_filtered[i])
        patients_list.append(patients_baseline[i])
        labels_list.append(labels[i])
        if i == 8:
            part_a = list(signals_filtered[i][:750])
            part_b = list(signals_filtered[i][2625:])
            
            signals_new[i] = part_a; 
            signals_new.append(part_b)
            
            patients_list[i] = patients_baseline[i]
            patients_list.append(patients_baseline[i])
            
            labels_list[i] = labels[i]
            labels_list.append(labels[i])
            del part_a
            del part_b
                
        if i == 10:
            part_a = list(signals_filtered[i][:8550]); part_b = list(signals_filtered[i][9950:13250]); part_c = list(signals_filtered[i][13875:])
            a = len(signals_new)-1; 
            signals_new[a]= part_a
            patients_list[a] = patients_baseline[i]
            labels_list[a] = labels[i]
            
            signals_new.append(part_b);patients_list.append(patients_baseline[i]); labels_list.append(labels[i])
            signals_new.append(part_c);patients_list.append(patients_baseline[i]); labels_list.append(labels[i])
            
            #signals_filtered[i] = np.array(part_a + part_b + part_c)
            
    return signals_new, patients_list,labels_list

# Calibrate signals

In [ ]:
# Normalize BP signals between SP and DP 
def calibrate_signals(BP, scale_factor_BP=40):
    scaled_BP = []
    #scaled_CVP = [],[]   scale_factor_CVP=8
    for i in range (0,len(BP)):
        scaled_BP.append(scale_factor_BP * np.array(BP[i]))
        #scaled_CVP.append(scale_factor_CVP * np.array(CVP[i]))
#         a = round(DP[i])
#         b = round(SP[i])
#         A = np.min(signal)
#         B = np.max(signal)
#         C = (a-b)/(A-B)
#         k = (C*A - a)/C
#         normalized = (signal-k)*C
    return scaled_BP  #, scaled_CVP


# Denoising + baseline correction functions

In [2]:
def denoise_signal(X, dwt_transform, cutoff_low, cutoff_high):
    coeffs = wavedec(X, dwt_transform)   # wavelet transform 'bior4.4'
    # scale 0 to cutoff_low 
    for ca in range(0,cutoff_low):
        coeffs[ca]=np.multiply(coeffs[ca],[0.0])
    # scale cutoff_high to end
    for ca in range(cutoff_high, len(coeffs)):
        coeffs[ca]=np.multiply(coeffs[ca],[0.0])
    Y = pywt.waverec(coeffs, dwt_transform) # inverse wavelet transform
    return Y 

def get_median_filter_width(sampling_rate, duration):
    res = int(sampling_rate*duration)
    res += ((res%2) - 1) # needs to be an odd number
    return res

def filter_signal(X,mfa):
    X0 = X  #read orignal signal
    for mi in range(0,len(mfa)):
        X0 = medfilt(X0,mfa[mi]) # apply median filter one by one on top of each other
    X0 = np.subtract(X,X0)  # finally subtract from orignal signal
    return X0

def denoise(ECG, fs):
    # Baseline correction
    ms_flt_array = [0.2,0.6]    #<-- length of baseline fitting filters (in seconds)
    mfa = np.zeros(len(ms_flt_array), dtype='int')
    for i in range(0, len(ms_flt_array)):
        mfa[i] = get_median_filter_width(fs,ms_flt_array[i])

    ECG_denoised = []
    for j in range(len(ECG)):
        # denoise signal using DWT 
        signal_den = denoise_signal(ECG[j],'bior4.4' , 1 , 12) #<--- trade off - the less the cutoff - the more R-peak morphology is lost
        signal_flt = filter_signal(signal_den,mfa) 
        
        ECG_denoised.append(signal_flt)
    return ECG_denoised

# GAN training

In [ ]:
def gradient_penalty(critic, labels, real, fake, device):
    BATCH_SIZE, H, W = real.shape

    alpha = torch.rand((BATCH_SIZE, 1, 1)).repeat(1, H, W).to(device)
    
    # Interpolate between real and fake data
    interpolates = real * alpha + fake * (1 - alpha)
    interpolates = autograd.Variable(interpolates, requires_grad=True)

    # Calculate critic scores
    mixed_scores = critic(interpolates,labels)

    # Take the gradient of the scores with respect to the input signals
    gradient = torch.autograd.grad(
        inputs=interpolates,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty    

In [ ]:
def catch_unique(list_in):
   # intilize an empty list
   unq_list = []

   # Check for elements
   for x in list_in:
      # check if exists in unq_list
      if x not in unq_list:
         unq_list.append(x)
         # print list
   for x in unq_list:
      print(x)

# Signals calibration

In [1]:
# Normalize BP signals between SP and DP 
def calibrate_signals(BP, CVP, scale_factor_BP=40, scale_factor_CVP=8):
    scaled_BP, scaled_CVP = [],[]
    for i in range (0,len(BP)):
        scaled_BP.append(scale_factor_BP * np.array(BP[i]))
        scaled_CVP.append(scale_factor_CVP * np.array(CVP[i]))
#         a = round(DP[i])
#         b = round(SP[i])
#         A = np.min(signal)
#         B = np.max(signal)
#         C = (a-b)/(A-B)
#         k = (C*A - a)/C
#         normalized = (signal-k)*C
    return scaled_BP, scaled_CVP
